<a href="https://colab.research.google.com/github/kridtapon/WFO-RVI-Coppock-Synergy/blob/main/WFO_RVI_Coppock_Synergy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate Relative Vigor Index (RVI)
def calculate_rvi(data, period=10):
    """
    Calculate Relative Vigor Index (RVI).
    """
    close_open_diff = data['Close'] - data['Open']
    high_low_diff = data['High'] - data['Low']

    numerator = close_open_diff.rolling(window=period).mean()
    denominator = high_low_diff.rolling(window=period).mean()

    rvi = numerator / denominator
    return rvi

# Function to calculate Coppock Curve
def calculate_coppock(data, wma_period=10, roc1_period=14, roc2_period=11):
    """
    Calculate Coppock Curve.
    """
    roc1 = data['Close'].pct_change(roc1_period) * 100
    roc2 = data['Close'].pct_change(roc2_period) * 100

    coppock = (roc1 + roc2).rolling(window=wma_period).mean()
    return coppock

# Walk-forward optimization with RVI and Coppock Curve
def walk_forward_optimization(df, start_year, end_year):
    results = []

    # Parameter grids for RVI and Coppock Curve
    rvi_period_range = range(5, 30, 2)
    wma_period_range = range(5, 30, 2)
    roc1_period_range = range(5, 30, 2)
    roc2_period_range = range(5, 30, 2)

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_start]

        best_params = None
        best_performance = -np.inf

        # Loop through parameter combinations
        for params in itertools.product(rvi_period_range, wma_period_range, roc1_period_range, roc2_period_range):
            rvi_period, wma_period, roc1_period, roc2_period = params

            # Calculate indicators on the training data
            train_data['RVI'] = calculate_rvi(train_data, period=rvi_period)
            train_data['Coppock'] = calculate_coppock(train_data, wma_period=wma_period, roc1_period=roc1_period, roc2_period=roc2_period)

            # Generate entry and exit signals
            entries = (train_data['RVI'] > 0) & (train_data['Coppock'] > 0)
            exits = (train_data['RVI'] < 0) & (train_data['Coppock'] < 0)

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = params

        # Test with the best parameters on the test data
        test_data['RVI'] = calculate_rvi(test_data, period=best_params[0])
        test_data['Coppock'] = calculate_coppock(test_data, wma_period=best_params[1], roc1_period=best_params[2], roc2_period=best_params[3])

        entries = (test_data['RVI'] > 0) & (test_data['Coppock'] > 0)
        exits = (test_data['RVI'] < 0) & (test_data['Coppock'] < 0)

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)

# Define the stock symbol and time period
symbol = 'APO'  # Example symbol
start_date = '2015-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Perform walk-forward optimization
results = walk_forward_optimization(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combined entry/exit signals
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    yearly_data['RVI'] = calculate_rvi(yearly_data, period=params[0])
    yearly_data['Coppock'] = calculate_coppock(yearly_data, wma_period=params[1], roc1_period=params[2], roc2_period=params[3])

    entries = (yearly_data['RVI'] > 0) & (yearly_data['Coppock'] > 0)
    exits = (yearly_data['RVI'] < 0) & (yearly_data['Coppock'] < 0)

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
<ipython-input-2-6690b0cdc536>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['RVI'] = calculate_rvi(train_data, period=rvi_period)
<ipython-input-2-6690b0cdc536>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Coppock'] = calculate_coppock(train_data, wma_period=wma_period, roc1_period=roc1_period, roc2_period=roc2_period)
<ipython-input-2-6690b0cdc536>:58: SettingWithCopyWarning: 
A value is trying to be set o


Walk-Forward Optimization Results:
   Year      Best_Params
0  2020  (5, 29, 15, 29)
1  2021    (5, 9, 9, 29)
2  2022   (5, 5, 25, 25)
3  2023     (5, 5, 7, 7)
4  2024    (9, 5, 7, 19)
5  2025     (9, 5, 7, 7)


<ipython-input-2-6690b0cdc536>:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yearly_data['RVI'] = calculate_rvi(yearly_data, period=params[0])
<ipython-input-2-6690b0cdc536>:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yearly_data['Coppock'] = calculate_coppock(yearly_data, wma_period=params[1], roc1_period=params[2], roc2_period=params[3])
<ipython-input-2-6690b0cdc536>:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           208481.259083
Total Return [%]                       108.481259
Benchmark Return [%]                   302.646983
Max Gross Exposure [%]                      100.0
Total Fees Paid                       6429.043722
Max Drawdown [%]                        34.770499
Max Drawdown Duration                       538.0
Total Trades                                   23
Total Closed Trades                            23
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            52.173913
Best Trade [%]                          46.660754
Worst Trade [%]                        -22.624513
Avg Winning Trade [%]                   15.011358
Avg Losing Trade [%]                     -7.46437


In [3]:
# Buy and Hold Performance Metrics
df_holding = df['Close']
pf = vbt.Portfolio.from_holding(df_holding, init_cash=100_000)
pf.stats()

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



,Close
Start,2020-01-02 00:00:00
End,2024-12-31 00:00:00
Period,1258
Start Value,100000.0
End Value,402646.983056
Total Return [%],302.646983
Benchmark Return [%],302.646983
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],53.479435
